In [9]:
import fsspec
from datetime import datetime,timedelta
from pyspark.sql import SparkSession
import json
from shapely import wkt
from shapely.geometry import Point
from pyspark.sql.functions import col, input_file_name, regexp_extract,first,to_timestamp,when,least,lit,round,year, month, hour, dayofweek, to_date,day,udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType,IntegerType
from notebookutils import mssparkutils

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 13, Finished, Available, Finished)

In [3]:
spark=SparkSession.builder.appName('air_ffill').getOrCreate()
source = "abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/aq"
checkpoint_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/8d633764-c954-4f64-a6ff-b52dde76bd20/Files/checkpoint/checkpoint.json"
zones_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/8d633764-c954-4f64-a6ff-b52dde76bd20/Tables/silver/taxi_zones"
account_name = "4906b11e-1e59-4869-9321-062a4696a2db"
account_host = "onelake.dfs.fabric.microsoft.com"

fs = fsspec.filesystem(
    "abfss",
    account_name=account_name,
    account_host=account_host
)

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 7, Finished, Available, Finished)

In [4]:
with fs.open(f"{source}/sensors/sensors.json") as f:
    metadata=json.load(f)
metadata_rows=[(k,v['latitude'],v['longitude']) for k,v in metadata.items()]
df_metadata = spark.createDataFrame(metadata_rows, ["sensor_id", "latitude", "longitude"])

with fs.open(checkpoint_path) as f:
    checkpoint=json.load(f)

zones=spark.read.format('delta').load(zones_path)
zones_list = [(row.BoroughID, wkt.loads(row.geometry)) for row in zones.collect()]

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 8, Finished, Available, Finished)

In [5]:
def latest_date():
    base_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/aq/hourly"
    years = [int(y.name) for y in mssparkutils.fs.ls(base_path)]
    year = max(years)
    months = [int(m.name) for m in mssparkutils.fs.ls(f"{base_path}/{year}")]
    month = max(months)
    days = [int(d.name) for d in mssparkutils.fs.ls(f"{base_path}/{year}/{str(month).zfill(2)}")]
    day = max(days)


    return (year, month, day, 0,0)

def generate_paths(start,end):
    if isinstance(start, str):
        start = datetime.fromisoformat(start)
    paths=[]
    curr=start
    if curr==end:
        return paths
    while curr<=end:
        paths.append(
            f"{source}/hourly/"
            f"{curr.year}/"
            f"{curr.month:02d}/"
            f"{curr.day:02d}/"
            f"{curr.hour:02d}/*/*.json"
        )
        curr+=timedelta(hours=1)
    return paths

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 9, Finished, Available, Finished)

In [6]:
def point_to_borough(lat, lon):
    pt = Point(lon, lat)
    for borough_id, poly in zones_list:
        if poly.contains(pt):
            return borough_id
    return None

point_to_borough_udf = udf(point_to_borough, IntegerType())

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 10, Finished, Available, Finished)

In [7]:
schema = StructType([
    StructField("value", DoubleType(), True),
    StructField("parameter", StructType([
        StructField("name", StringType(), True)
    ]), True),
    StructField("period", StructType([
        StructField("datetime_from", StructType([
            StructField("local", StringType(), True)
        ]), True)
    ]), True),
    StructField("_corrupt_record", StringType(), True)
])
def read_file(paths,df_metadata,schema):
    df_raw = (
        spark.read
            .option("multiLine", True)
            .schema(schema)
            .json(paths)
            .withColumn("file_path", input_file_name())
            .withColumn("sensor_id", regexp_extract("file_path", r"/([^/]+)/[^/]+\.json$", 1))
            .drop("file_path")
    ).select(
            "sensor_id",
            to_timestamp(
            col("period.datetime_from.local")).alias("timestamp_local"),
            col("parameter.name").alias("parameter_name"),
            col("value").alias("measurement_value")
        ).groupBy("sensor_id", "timestamp_local").pivot("parameter_name", ["pm25"]).agg(first("measurement_value"))
    
    df=df_raw.join(df_metadata,on='sensor_id',how='left')
    df=df.fillna(0)
    for p in ["pm25"]:
        df = df.withColumn(
            p,
            when(col(p) < 0, 0).otherwise(col(p))
        )

    df=(df.groupby("latitude", "longitude", "timestamp_local")
    .agg(
            first("pm25").alias("pm25")
    ))
    df=df.withColumn("LocationID",point_to_borough_udf(col('latitude'),col('longitude')))
    df = df.drop("latitude", "longitude")
    df = df.withColumn("year", year("timestamp_local")) \
        .withColumn("month", month("timestamp_local"))
    df.write \
        .format("delta") \
        .mode("append") \
        .partitionBy("year", "month") \
        .saveAsTable("silver.air_quality")

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 11, Finished, Available, Finished)

In [8]:
to_silver = checkpoint.get("air_quality", datetime(2025, 12, 16, 0, 0))
paths=generate_paths(to_silver,datetime(*latest_date()))
if paths:
    read_file(paths=paths,df_metadata=df_metadata,schema=schema)



StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, 12, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o6787.saveAsTable.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.awaitShuffleMapStage$1(DeltaOptimizedWriterExec.scala:157)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.getShuffleStats(DeltaOptimizedWriterExec.scala:162)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.computeBins(DeltaOptimizedWriterExec.scala:104)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.doExecute(DeltaOptimizedWriterExec.scala:178)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:220)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:271)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	

StatementMeta(, aec22929-69fe-4ce4-8cd3-842ee5b20fda, -1, Cancelled, , Cancelled)